In [439]:
import pandas as pd
import re
import pickle
from tqdm import tqdm
from time import sleep
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [ ]:
df = pd.read_csv("data/wikihowSep.csv")
df = df.dropna()
tasks = df.title.unique().tolist()
# remove number from tasks
tasks = list(set([re.sub(r'\W+', ' ', i[:-1].lower()) if str.isnumeric(i[-1]) else re.sub(r'\W+', ' ', i.lower()) for i in tasks]))


In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }

    params = urllib.parse.urlencode({})
    
    try:
        conn = http.client.HTTPSConnection('api.msturing.org')
#         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
        conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
        response = conn.getresponse()
        data = response.read()
        data = json.loads(data)
        conn.close()
    except Exception as e:
        print(data)
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

In [234]:
def load_dict(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [255]:
tmp1 = load_dict("wikihow_2_ms_encoded_vector/0-50")
tmp2 = load_dict("wikihow_2_ms_encoded_vector/8650-8700")

In [440]:
import glob
files = []
for file in glob.glob("wikihow_2_ms_encoded_vector/*"):
    files.append(file)

In [441]:
ranges = [(i, i+50) for i in range(0, len(tasks), 50)]

In [442]:
finish = []
for f in files:
    try:
        name = f.split("/")[1].split("-")
        finish.append((int(name[0]), int(name[1])))
    except:
        continue
        

In [443]:
todo = list(set.difference(set(ranges), set(finish)))

In [444]:
for i in tqdm(todo, total=len(todo)):
    try:
        data = getVectors(tasks[i[0]:i[1]])
        with open("wikihow_2_ms_encoded_vector/%s-%s" % (str(i[0]), str(i[1])), 'wb') as f:
            pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    except:
        sleep(60)

b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF09FA63DA6 failed due to timeout'
b'All 1 requests failed. Shared memory request 637084800854294908 to turing.agiencoderv4_02082019:131992289164512738 instance 3 timed out'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF09FA63DA6 failed due to timeout'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS1PPF09FA63DA6 failed due to timeout'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS2PPF062D4B1F3 failed due to timeout'
b'All 1 requests failed. Shared memory request 637086578503563248 to turing.agiencoderv4_02082019:131992289164512738 instance 3 timed out'
b'All 1 requests failed. Shared memory request 637086578503563256 to turing.agiencoderv4_02082019:131992289164512738 instance 3 timed out'
b'All 1 requests failed. Invoking turing.agiencoderv4_02082019:131992289164512738 on DS

KeyboardInterrupt: 

In [ ]:
task_embs = []
task_names = []
for i in finish[1:]:
    emb = load_dict("wikihow_2_ms_encoded_vector/%d-%d" % (i[0], i[1]))
    task_names.extend([j for j in emb])    
    emb = [emb[j] for j in emb][0]
    task_embs.append(emb)


In [307]:
from sklearn.metrics.pairwise import cosine_similarity


In [337]:
online_emb = load_dict("wikihow_2_ms_encoded_vector/%d-%d" % (finish[0][0], finish[0][1]))
online_emb = [online_emb[i] for i in online_emb][0]


In [392]:
scores = []
for t in task_embs:
    scores.append(cosine_similarity([onLine_emb], [t])[0])

In [407]:
sorted_idx = [i[0] for i in sorted(enumerate(scores), key=lambda x:x[1])]
sorted_idx = sorted_idx[::-1]

In [417]:
for i in sorted_idx[:100]:
    print(task_names[i])

how to play tennis
how to diagnose mitral valve disease in cavalier king charles spaniels
how to find a physician job
how to sign up for flipnote hatena without sending in the permission form
how to pick a top team for mario strikers charged football
how to make a cauldron in minecraft
how to file an flsa complaint
how to open links in a new tab on an iphone
how to make a maple shortbow in runescape
how to have a 5 minute mohawk
how to clean a driveway
how to make a smoky cheese ball
how to make learning fun
how to lubricate a garage door
how to paint with dandelions
how to grow perennial vegetables
how to prepare a shrub bed
how to make laab moo
how to make a chilli pepper garden spray
how to resolve livestock collision disputes
how to uninstall steam without additional programs
how to look awesome at a school dance girls 
how to grill donuts
how to keep a cat in diabetic remission
how to integrate by differentiating under the integral
how to deal with bullies at school
how to get goo

In [458]:
data = getVectors(["book flight ticket", "online", "play tennis online"])

In [453]:
cosine_similarity([data["play tennis"]], [data["online"]])

array([[0.29493439]])

In [454]:
cosine_similarity([data["play tennis online"]], [data["play tennis"]])

array([[0.89417903]])

In [455]:
cosine_similarity([data["play tennis online"]], [data["online"]])

array([[0.65585031]])

In [459]:
cosine_similarity([data["book flight ticket"]], [data["online"]])

array([[0.30638924]])